In [1]:
import tensorflow as tf
import re

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# create config to use both gpus on wickerman machine:
# 1. Nvidia A40 (25GB memory allocation)
# 2. Nvidia RTX 3060 (11GB memory allocation)

gpus = tf.config.list_physical_devices('GPU')

# tf.config.set_visible_devices(gpus[0], 'GPU') # use Use Nvidia A40 only

# log_dev_conf_a40 = tf.config.LogicalDeviceConfiguration(
#     memory_limit=25*1024 # 25 GB allocation for a40 GPU
# )

# tf.config.set_logical_device_configuration(
#     gpus[0], # select GPU_0, i.e., Nvidia A40
#     [log_dev_conf_a40] # apply 25GB config
# )

tf.config.set_visible_devices(gpus[1], 'GPU') # use RTX3060 only

log_dev_conf_rtx3060 = tf.config.LogicalDeviceConfiguration(
    memory_limit=12*1024 # 11 GB allocation for rtx3060 GPU
)

tf.config.set_logical_device_configuration(
    gpus[1], # select GPU_1, i.e., Nvidia rtx3060
    [log_dev_conf_rtx3060] # apply 11GB config
)

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")

2024-08-26 16:44:05.516176: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-26 16:44:05.516198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-26 16:44:05.516974: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-26 16:44:05.520905: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-26 16:44:06.074720: W tensorflow/compiler/tf2

2 Physical GPUs, 1 Logical GPU


2024-08-26 16:44:06.785111: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-26 16:44:06.786403: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-26 16:44:06.806806: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [13]:
import json
import os

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Progbar
from tensorflow.keras.metrics import Mean
from pqdm.threads import pqdm
from tqdm import tqdm

In [16]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration

# model_id = "google-t5/t5-base"
model_id = 'google/flan-t5-base'

# with strategy.scope():
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = TFT5ForConditionalGeneration.from_pretrained(model_id)

# tokenizer = AutoTokenizer.from_pretrained("models/v2")
# model = TFT5ForConditionalGeneration.from_pretrained("models/v4")
# model = TFT5ForConditionalGeneration.from_pretrained("ckpts/v4/ckpt-0")


2024-08-26 18:39:21.020980: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [1]:
def load_jsonl(path):
    with open(path) as input_file:
        lines = list(map(json.loads, input_file))

    return lines

def load_data(root_path):
    files = os.listdir(root_path)
    dataset= {}

    for filename in files:
        filepath = root_path + f'/{filename}'

        dataset[filename] = load_jsonl(filepath)

    return dataset

In [ ]:
dataset = load_data('data/L1')

In [18]:
models = ['llama3.1', 'gemma2:2b', 'gemma2']

test_from = 'attrebute_val'
train_from = 'attrebute_train'
target_from = 'attrebute_test'

train_inputs = train_from + '.data'
train_labels = train_from + '.solution'

test_inputs = test_from + '.data'
test_labels = test_from + '.solution'

target_inputs = target_from + '.data'

test_inputs = dataset[test_inputs]
test_labels = dataset[test_labels]
train_inputs = dataset[train_inputs]
train_labels = dataset[train_labels]
target_inputs = dataset[target_inputs]

1/1 [==============================] - 0s 50ms/step


In [21]:
SEQ_LEN=250
prompt = {
    "label": "The product belongs to these categories:\nBrand: {details_Brand}\nLevel 0: {L0_category}\nLevel 1: {L1_category}\nLevel 2: {L2_category}\nLevel 3: {L3_category}\nLevel 4: {L4_category}",
    "input": "Categories the product into 5 levels of categories and identify the brand:\nTitle: {title}\nStore:{store}\nManufacturer:{details_Manufacturer}"
}

def make_prompt(data):
    if 'title' in data:
        input_type = 'input'
    else:
        input_type = 'label'

    prompt = {
        "label": "The product belongs to these categories:\nBrand: {details_Brand}\nLevel 0: {L0_category}\nLevel 1: {L1_category}\nLevel 2: {L2_category}\nLevel 3: {L3_category}\nLevel 4: {L4_category}",
        "input": "Categorise the product into 5 levels of categories and identify the brand:\nTitle: {title}\nStore:{store}\nManufacturer:{details_Manufacturer}"
    }

    return prompt[input_type].format(**data)

def batch_make_prompt(data):
    return [make_prompt(item) for item in data]
# def tokenize()

(2, 256, 30522)

In [ ]:
train_input_prompts = pqdm(train_inputs, make_prompt, n_jobs=5)
train_label_prompts = pqdm(train_labels, make_prompt, n_jobs=5)
test_input_prompts = pqdm(test_inputs, make_prompt, n_jobs=5)
test_label_prompts = pqdm(test_labels, make_prompt, n_jobs=5)

In [ ]:
def train_model(inputs, labels, return_output=False):
    inputs = [item.decode() for item in inputs]
    labels = [item.decode() for item in labels]

    encoding = tokenizer(
        inputs,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )

    input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

    target_encoding = tokenizer(
        labels,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )

    labels = target_encoding.input_ids
    labels[labels == tokenizer.pad_token_id] = -100

    preds = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

    if not return_output:
        return preds.loss
    
    return preds, preds.loss

def predict(inputs):
    

    encoding = tokenizer.batch_encode_plus(
        inputs,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )

    input_ids, attention_mask = encoding.input_ids, encoding.attention_mask
    outputs = model.generate(input_ids, max_new_tokens=SEQ_LEN)

    return outputs